In [1]:
import folium 

m = folium.Map(location= [45, 0], zoom_start=4)
m

In [2]:
import json

with open('dummy.json') as json_file:
    data = json.load(json_file)
print("Data Loaded.")

Data Loaded.


In [3]:
print("There is", len(data['locations']), "different GPS locations")

There is 2112 different GPS locations


In [4]:
from math import sin, cos, sqrt, atan2, radians, floor, log2

In [5]:
locations = []
for loc in data['locations']:
    lat1 = loc['latitudeE7']*10**(-7)
    lon1 = loc['longitudeE7']*10**(-7)
    locations.append((lat1, lon1))
len(locations)

2112

In [6]:
R = 6373.0 ##average earth radius.
def distance(a, b):
    '''Returns the distance from point A to point B in kilometers.'''
    (lat1, lon1) = radians(a[0]), radians(a[1])
    (lat2, lon2) = radians(b[0]), radians(b[1])
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c

    return distance

In [7]:
PRECISION = 0.05 ##Points which are less than 5 km apart need to be merged.

In [8]:
import rtree

In [9]:
def sparse_subset(points, r):
    """Return a maximal list of elements of points such that no pairs of
    points in the result have distance less than r.

    """ ##thxx stackexchange
    result = []
    index = rtree.index.Index()
    for i, p in enumerate(points):
        px, py = p
        nearby = index.intersection((px - r, py - r, px + r, py + r))
        if all(distance(p, points[j]) >= r for j in nearby):
            result.append(p)
            index.insert(i, (px, py, px, py))
    return result

In [10]:
s = sparse_subset(locations, PRECISION)
print("Data has been reduced from",len(locations),"points to",len(s),"points.")

Data has been reduced from 2112 points to 36 points.


In [11]:
for l in s:
    folium.Circle(
    radius=PRECISION,
    location= l,
    color='crimson',
    fill=False,
).add_to(m)
m